In [1]:
import os, six, time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, utils, layers, models, optimizers, losses, metrics

from tensorflow.keras import backend as K
import  urllib

seed = 13
tf.random.set_seed(seed)
np.random.seed(seed)

In [16]:
batch_size = 64
epochs = 3

data_dir = os.path.join(".","data")
SOURCE_TRAIN_URL = "https://raw.githubusercontent.com/random-forests/tensorflow-workshop/master/archive/extras/colorbot/data/train.csv"
SOURCE_TEST_URL = "https://raw.githubusercontent.com/random-forests/tensorflow-workshop/master/archive/extras/colorbot/data/test.csv"

def color_dataset():
    #os.makedirs(data_dir, exists_ok=True)
    train_data = load_dataset(data_dir=data_dir, url=SOURCE_TRAIN_URL, batch_size=batch_size)
    eval_data = load_dataset(data_dir=data_dir, url=SOURCE_TEST_URL, batch_size=batch_size)
    
    return train_data, eval_data

def parse(line):
    # Each line of the dataset is comma-separated and formatted as
    #    color_name, r, g, b
    # `items` is a list [color_name, r, g, b].
    items = tf.strings.split([line], sep=",").values
    rgb = tf.strings.to_number(items[1:], out_type=tf.float32) / 255.
    color_name = items[0]
    chars = tf.one_hot(tf.io.decode_raw(color_name, tf.uint8), depth=256)
    length = tf.cast(tf.shape(chars)[0], dtype=tf.int64)
    return rgb, chars, length

def load_dataset(data_dir, url, batch_size):
    ## basename()   用于去掉目录的路径，只返回文件名
    path = maybe_download(os.path.basename(url), data_dir, url)
    # This chain of commands loads our data by:
    #   1. skipping the header; (.skip(1))
    #   2. parsing the subsequent lines; (.map(parse))
    #   3. shuffling the data; (.shuffle(...))
    #   3. grouping the data into padded batches (.padded_batch(...)).
    dataset = tf.data.TextLineDataset(path).skip(1).map(parse).shuffle(
        buffer_size=10000).padded_batch(batch_size, padded_shapes=([None], [None,None], []))    
    return dataset    
    
def maybe_download(filename, work_directory, source_url):
    if not tf.io.gfile.exists(work_directory):
        tf.io.gfile.makedirs(work_directory)
    filepath = os.path.join(work_directory, filename)
    if not tf.io.gfile.exists(filepath):
        temp_data, _ = urllib.request.urlretrieve(source_url)
        tf.io.gfile.copy(temp_data, filepath)
        with tf.io.gfile.GFile(filepath) as f:
            size = f.size()
            print("Successfully downloaded", filename, size, "bytes.")
    print(filepath,"exists")
    return filepath

train_data, test_data = color_dataset()

.\data\train.csv exists
.\data\test.csv exists


In [112]:
#del RNNColorbot
#del RNNCell
class RNNColorbot(models.Model):
    def __init__(self, rnn_cell_sizes, label_dimension, keep_prob):
        super(RNNColorbot, self).__init__()
        self.rnn_cell_sizes = rnn_cell_sizes
        self.label_dimension = label_dimension
        self.keep_prob = keep_prob
        self.model = self.create_model()
        
    def call(self, inputs):        
        outputs = self.model(inputs)
        return outputs
    
    def create_model(self):
        model = models.Sequential()        
        cells = [RNNCell(units) for units in self.rnn_cell_sizes]        
        model.add(layers.RNN(cells,input_shape=(22,256)))
        model.add(layers.Dropout(rate=(1-self.keep_prob)))        
        model.add(layers.Dense(units=self.label_dimension, activation="relu"))
        return model
    
class RNNCell(layers.Layer):
    def __init__(self, units, **kwargs):
        super(RNNCell, self).__init__(**kwargs)
        self.units = units
        self.state_size = units
        
    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer="uniform", name="kernel")
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units),
                                      initializer="uniform", name="recurrent_kernel")
        self.build = True
        
    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        return output, [output]


In [113]:
rnn_cell_sizes = [256, 128]
model = RNNColorbot(
        rnn_cell_sizes=rnn_cell_sizes,
        label_dimension=3,
        keep_prob=0.7)
model.compile(optimizer=optimizers.Adam(0.05),
                  loss="mean_squared_error",
                  metrics=["accuracy"])

In [111]:
epochs = 1
for step, (labels, chars, sequence_length) in enumerate(train_data):
    #chars = tf.transpose(chars, [1,0,2])
    #batch_size = int(chars.shape[1])
    model.fit(chars, labels, batch_size=batch_size,epochs=epochs, verbose=2) #
    

Train on 64 samples
64/64 - 1s - loss: nan - accuracy: 0.6094
Train on 64 samples
64/64 - 1s - loss: nan - accuracy: 0.5156
Train on 64 samples
64/64 - 1s - loss: nan - accuracy: 0.5938
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.6406
Train on 64 samples
64/64 - 1s - loss: nan - accuracy: 0.5625
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.5938
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.5625
Train on 64 samples
64/64 - 1s - loss: nan - accuracy: 0.4844
Train on 64 samples
64/64 - 1s - loss: nan - accuracy: 0.5312
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.6562
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.7188
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.5938
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.5938
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.5938
Train on 64 samples
64/64 - 0s - loss: nan - accuracy: 0.5938
Train on 64 samples
64/64 - 1s - loss: nan - accuracy: 0.6562
Train on